In [1]:
from google.colab import drive
import os
import glob
import pandas as pd

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/data-science-veiculos-poluicao'
os.chdir(folder_path)

print(os.listdir())

# Get a list of all CSV files in the 'iema' directory
csv_files = glob.glob('senatran/ano_modelo/*.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['senatran', 'iema', 'iema-agg', 'senatran-agg', 'iema-2.ipynb', 'poluentes_anual.csv', 'detran', 'ARQUIVO', 'ano_modelo.ipynb', 'idades_veiculos_anual.csv', 'panel_idade_poluicao_combusteiveis.ipynb', 'anp', 'combustiveis.ipynb', 'combustiveis.csv']


In [2]:
# Read all CSV files into a single DataFrame
all_data = []
for file in csv_files:
  try:
    df = pd.read_csv(file)
    # Extract the year from the filename (assuming format "frota-{year}.csv")
    year = int(file.split('/')[-1].split('-')[-1].split('.')[0])
    df['year'] = year
    all_data.append(df)
  except Exception as e:
    print(f"Error reading {file}: {e}")

df = pd.concat(all_data, ignore_index=True)

# Display the first few rows of the combined DataFrame
display(df.head())

,UF,Municipio,Modelo,Fabricacao,qtd,year
0,ACRE,ACRELANDIA,1967,1967,1,2015
1,ACRE,ACRELANDIA,1968,1968,1,2015
2,ACRE,ACRELANDIA,1971,1971,1,2015
3,ACRE,ACRELANDIA,1972,1972,2,2015
4,ACRE,ACRELANDIA,1973,1973,4,2015


In [3]:
sp_df = df[df['UF'] == 'SAO PAULO'].copy() # Create a copy to avoid SettingWithCopyWarning

sp_df['Modelo'] = pd.to_numeric(sp_df['Modelo'], errors='coerce')
sp_df['Fabricacao'] = pd.to_numeric(sp_df['Fabricacao'], errors='coerce')
sp_df['qtd'] = pd.to_numeric(sp_df['qtd'], errors='coerce')

sp_df.dropna(subset=['Modelo', 'Fabricacao'], inplace=True)

sp_df['Modelo'] = sp_df['Modelo'].astype(int)
sp_df['Fabricacao'] = sp_df['Fabricacao'].astype(int)
sp_df['qtd'] = sp_df['qtd'].astype(int)

display(sp_df.head())

,UF,Municipio,Modelo,Fabricacao,qtd,year
451433,SAO PAULO,ADAMANTINA,1900,1979,1,2015
451434,SAO PAULO,ADAMANTINA,1929,1929,2,2015
451435,SAO PAULO,ADAMANTINA,1948,1948,3,2015
451436,SAO PAULO,ADAMANTINA,1951,1951,4,2015
451437,SAO PAULO,ADAMANTINA,1952,1952,1,2015


In [4]:
sp_df = sp_df[sp_df['qtd'] >= 10][['year', 'Municipio', 'Fabricacao', 'qtd']]
sp_df = sp_df.rename(columns={'Municipio': 'cidade'})
display(sp_df.head())

,year,cidade,Fabricacao,qtd
451448,2015,ADAMANTINA,1965,14
451449,2015,ADAMANTINA,1966,18
451451,2015,ADAMANTINA,1968,45
451453,2015,ADAMANTINA,1969,29
451454,2015,ADAMANTINA,1970,48


In [5]:
# Import the annual_december_avg.csv file
annual_avg_df = pd.read_csv('poluentes_anual.csv')

# Get unique values from the 'cidade' column
unique_cidades = annual_avg_df['cidade'].unique()

# Display the unique values
print("Unique values in 'cidade' column:")
print(unique_cidades)

Unique values in 'cidade' column:
['AMERICANA' 'ARACATUBA' 'ARARAQUARA' 'BAURU' 'CAMPINAS' 'CARAPICUIBA'
 'CATANDUVA' 'CORDEIROPOLIS' 'CUBATAO' 'DIADEMA' 'FRANCA' 'GUARATINGUETA'
 'GUARUJA' 'GUARULHOS' 'JABOTICABAL' 'JACAREI' 'JAU' 'JUNDIAI' 'LIMEIRA'
 'MARILIA' 'MAUA' 'MOGI DAS CRUZES' 'OSASCO' 'PAULINIA' 'PIRACICABA'
 'PRESIDENTE PRUDENTE' 'RIBEIRAO PRETO' 'RIO CLARO' 'SANTA GERTRUDES'
 'SANTO ANDRE' 'SANTOS' 'SAO BERNARDO DO CAMPO' 'SAO CAETANO DO SUL'
 'SAO JOSE DO RIO PRETO' 'SAO JOSE DOS CAMPOS' 'SAO PAULO' 'SAO SEBASTIAO'
 'SOROCABA' 'TABOAO DA SERRA' 'TATUI' 'TAUBATE']


In [6]:
# Filter sp_df to keep rows where 'Municipio' is in unique_cidades
sp_df = sp_df[sp_df['cidade'].isin(unique_cidades)]

# Display the first few rows of the filtered DataFrame
display(sp_df.head())

,year,cidade,Fabricacao,qtd
453404,2015,AMERICANA,1951,19
453410,2015,AMERICANA,1957,12
453411,2015,AMERICANA,1958,14
453412,2015,AMERICANA,1959,15
453413,2015,AMERICANA,1960,16


# Task
Create a new dataframe based on the dataframe `sp_df` that has columns `year`, `cidade`, and columns representing age ranges of vehicles ("0-1 years", "1-2 years", ..., "9-10 years", "older than 10 years"), where the values are the sum of `qtd` for vehicles falling into each age range, calculated as `year` - `Fabricacao`.

## Calculate vehicle age

### Subtask:
Create a new column in `sp_df` representing the age of each vehicle (year - Fabricacao).


**Reasoning**:
Create a new column 'vehicle_age' by subtracting 'Fabricacao' from 'year' in the sp_df DataFrame.



In [7]:
sp_df['vehicle_age'] = sp_df['year'] - sp_df['Fabricacao']
display(sp_df.head())

,year,cidade,Fabricacao,qtd,vehicle_age
453404,2015,AMERICANA,1951,19,64
453410,2015,AMERICANA,1957,12,58
453411,2015,AMERICANA,1958,14,57
453412,2015,AMERICANA,1959,15,56
453413,2015,AMERICANA,1960,16,55


## Define age ranges

### Subtask:
Create bins for the desired age ranges (0-1, 1-2, ..., 9-10, >10 years).


**Reasoning**:
Define the bin edges and labels for the vehicle age ranges.



In [8]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, float('inf')]
labels = ['0a1', '1a2', '2a3', '3a4', '4a5', '5a6', '6a7', '7a8', '8a9', '9a10', 'acima10']

## Categorize vehicles by age range

### Subtask:
Assign each vehicle to an age range based on its calculated age.


**Reasoning**:
Categorize the 'vehicle_age' column into age ranges using the defined bins and labels and display the updated dataframe.



In [9]:
sp_df['age_range'] = pd.cut(sp_df['vehicle_age'], bins=bins, labels=labels, right=False, include_lowest=True)
display(sp_df.head())

,year,cidade,Fabricacao,qtd,vehicle_age,age_range
453404,2015,AMERICANA,1951,19,64,acima10
453410,2015,AMERICANA,1957,12,58,acima10
453411,2015,AMERICANA,1958,14,57,acima10
453412,2015,AMERICANA,1959,15,56,acima10
453413,2015,AMERICANA,1960,16,55,acima10


## Group and aggregate data

### Subtask:
Group the data by 'year' and 'cidade', and calculate the sum of 'qtd' for each age range within each group.


**Reasoning**:
Group the DataFrame by year, city, and age range and sum the quantity for each group.



In [10]:
sp_df_agg = sp_df.groupby(['year', 'cidade', 'age_range'])['qtd'].sum().reset_index()
display(sp_df_agg.head())

/tmp/ipython-input-1132880117.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sp_df_agg = sp_df.groupby(['year', 'cidade', 'age_range'])['qtd'].sum().reset_index()


,year,cidade,age_range,qtd
0,2015,AMERICANA,0a1,4304
1,2015,AMERICANA,1a2,7780
2,2015,AMERICANA,2a3,9611
3,2015,AMERICANA,3a4,8857
4,2015,AMERICANA,4a5,9712


## Reshape the dataframe

### Subtask:
Pivot or unstack the aggregated data to have age ranges as columns.


**Reasoning**:
Pivot the aggregated data to have age ranges as columns and fill missing values with 0.



In [11]:
sp_df_pivot = sp_df_agg.pivot_table(index=['cidade', 'year'], columns='age_range', values='qtd', fill_value=0)
display(sp_df_pivot.head())

/tmp/ipython-input-864905571.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  sp_df_pivot = sp_df_agg.pivot_table(index=['cidade', 'year'], columns='age_range', values='qtd', fill_value=0)


age_range          0a1     1a2     2a3     3a4     4a5     5a6     6a7  \
cidade    year                                                           
AMERICANA 2015  4304.0  7780.0  9611.0  8857.0  9712.0  9673.0  8366.0   
          2016  3328.0  5680.0  7974.0  9699.0  8874.0  9737.0  9548.0   
          2017  4075.0  4377.0  5959.0  8101.0  9672.0  8862.0  9690.0   
          2018  5157.0  5203.0  4632.0  6017.0  8057.0  9631.0  8789.0   
          2019  5229.0  6631.0  5564.0  4704.0  6034.0  8038.0  9444.0   

age_range          7a8     8a9    9a10   acima10  
cidade    year                                    
AMERICANA 2015  9885.0  9279.0  7195.0   80666.0  
          2016  8403.0  9852.0  9244.0   86724.0  
          2017  9528.0  8305.0  9716.0   94948.0  
          2018  9661.0  9442.0  8264.0  103249.0  
          2019  8691.0  9524.0  9232.0  110020.0

## Clean up and finalize

### Subtask:
Rename columns and ensure the DataFrame is in the desired format.


**Reasoning**:
Reset the index of the pivoted DataFrame and display the head to finalize the data structure.



In [12]:
sp_df_pivot = sp_df_pivot.reset_index()
display(sp_df_pivot.head())

age_range,cidade,year,0a1,1a2,2a3,3a4,4a5,5a6,6a7,7a8,8a9,9a10,acima10
0,AMERICANA,2015,4304.0,7780.0,9611.0,8857.0,9712.0,9673.0,8366.0,9885.0,9279.0,7195.0,80666.0
1,AMERICANA,2016,3328.0,5680.0,7974.0,9699.0,8874.0,9737.0,9548.0,8403.0,9852.0,9244.0,86724.0
2,AMERICANA,2017,4075.0,4377.0,5959.0,8101.0,9672.0,8862.0,9690.0,9528.0,8305.0,9716.0,94948.0
3,AMERICANA,2018,5157.0,5203.0,4632.0,6017.0,8057.0,9631.0,8789.0,9661.0,9442.0,8264.0,103249.0
4,AMERICANA,2019,5229.0,6631.0,5564.0,4704.0,6034.0,8038.0,9444.0,8691.0,9524.0,9232.0,110020.0


**Reasoning**:
The columns have a multi-level index after pivoting and resetting the index. Flatten the column names to a single level.



In [13]:
sp_df_pivot.columns = sp_df_pivot.columns.get_level_values(0)
sp_df_pivot = sp_df_pivot.rename(columns={'year': 'ano'})
display(sp_df_pivot.head())

age_range,cidade,ano,0a1,1a2,2a3,3a4,4a5,5a6,6a7,7a8,8a9,9a10,acima10
0,AMERICANA,2015,4304.0,7780.0,9611.0,8857.0,9712.0,9673.0,8366.0,9885.0,9279.0,7195.0,80666.0
1,AMERICANA,2016,3328.0,5680.0,7974.0,9699.0,8874.0,9737.0,9548.0,8403.0,9852.0,9244.0,86724.0
2,AMERICANA,2017,4075.0,4377.0,5959.0,8101.0,9672.0,8862.0,9690.0,9528.0,8305.0,9716.0,94948.0
3,AMERICANA,2018,5157.0,5203.0,4632.0,6017.0,8057.0,9631.0,8789.0,9661.0,9442.0,8264.0,103249.0
4,AMERICANA,2019,5229.0,6631.0,5564.0,4704.0,6034.0,8038.0,9444.0,8691.0,9524.0,9232.0,110020.0


**Reasoning**:
The column names are now flattened and the DataFrame is in the desired format. Display the first few rows to confirm the structure.



In [14]:
display(sp_df_pivot.head())

age_range,cidade,ano,0a1,1a2,2a3,3a4,4a5,5a6,6a7,7a8,8a9,9a10,acima10
0,AMERICANA,2015,4304.0,7780.0,9611.0,8857.0,9712.0,9673.0,8366.0,9885.0,9279.0,7195.0,80666.0
1,AMERICANA,2016,3328.0,5680.0,7974.0,9699.0,8874.0,9737.0,9548.0,8403.0,9852.0,9244.0,86724.0
2,AMERICANA,2017,4075.0,4377.0,5959.0,8101.0,9672.0,8862.0,9690.0,9528.0,8305.0,9716.0,94948.0
3,AMERICANA,2018,5157.0,5203.0,4632.0,6017.0,8057.0,9631.0,8789.0,9661.0,9442.0,8264.0,103249.0
4,AMERICANA,2019,5229.0,6631.0,5564.0,4704.0,6034.0,8038.0,9444.0,8691.0,9524.0,9232.0,110020.0


## Summary:

### Data Analysis Key Findings

*   A new column `vehicle_age` was successfully added to the `sp_df` DataFrame by subtracting `Fabricacao` from `year`.
*   Vehicles were categorized into specific age ranges (0-1 years, 1-2 years, ..., older than 10 years) based on their calculated `vehicle_age`.
*   The data was successfully grouped by `year`, `cidade`, and `age_range`, and the sum of `qtd` was calculated for each group.
*   The aggregated data was reshaped into a pivot table with `year` and `cidade` as the index and age ranges as columns, with `fill_value=0` for missing combinations.
*   The index of the pivoted DataFrame was reset, and the multi-level column names resulting from the pivot operation were flattened to a single level.

### Insights or Next Steps

*   The resulting DataFrame `sp_df_pivot` provides a clear overview of the distribution of vehicle quantities across different age ranges for each year and city.
*   This structured data is ready for further analysis, such as trend analysis of vehicle age over time per city or comparison of age distributions between different cities.


In [15]:
from google.colab import files

sp_df_pivot.to_csv('idades_veiculos_anual.csv', index=False)

#files.download('idades_veiculos_anual.csv')